In [171]:
#Dependencies
import os
import csv
import pandas as pd



#Paths
filename = "Resources/purchase_data.csv"
output_path = os.path.join("Output", "GenderAnalysis.csv")
output_path2 = os.path.join("Output", "AgeAnalysis.csv")


#Initialize Variables
names = []
ages = []
genders = []
itemids = []
itemnames = []
prices = []


males = 0
females = 0
other = 0 
malep = 0.0
femalep = 0.0
otherp = 0.0

uniqueplayers = []
uniqueitems = []


totalsales = 0
totalplayers = 0

uniqueitemsnum = 0
totalrevenue = 0.0
avgprice = 0.0



#Create Dataframes
df = pd.read_csv(filename, header=0, index_col=0)
df2 = df.copy()
df3 = df.copy()



#Create Dataframe Without Repeating Players
df2 = df2.drop_duplicates(subset=['SN'])



#Dataframe Load Columns
names = df["SN"]
ages = df["Age"]
genders = df2["Gender"]
itemids = df["Item ID"]
itemnames = df["Item Name"]
prices = df["Price"]



#Unique Names & Items
uniqueplayers = pd.Series(names).unique()  
uniqueplayers.sort()
uniqueitems = pd.Series(itemnames).unique()
uniqueitems.sort()



#Sales, Player, Revenue Totals & Average Price
totalsales = len(df.index)
totalplayers = len(df2)
uniqueitemsnum = len(uniqueitems)
totalrevenue = prices.sum()
avgprice = totalrevenue / totalsales



#Summary Gender Demographics (No Repeat Players)
genderlist = pd.Series(genders, dtype="string")
males = sum(genderlist.str.count('Male'))
females = sum(genderlist.str.count('Female'))
other = totalplayers - males - females
malep = (males / totalplayers) * 100
femalep = (females / totalplayers) * 100
otherp = (other / totalplayers) * 100


#Sort Sales By Gender 

#Males 
isMale = df['Gender']=='Male'
df_isMale = df[isMale]
malesales = len(df_isMale)
maleavg = (df_isMale['Price'].sum() / malesales)
malerevenue = df_isMale['Price'].sum()

males1 = df_isMale.sort_values(by=['SN'])
uniquemales2 = pd.Series(df_isMale['SN']).unique()   
uniquemales2.sort()
malenames = len(uniquemales2)

males1 = males1.drop(columns=['Age', 'Gender', 'Item ID', 'Item Name'])
males1 = males1.reset_index(drop=True)
maleprice = pd.Series(males1["Price"].values, index=males1['SN'])
maleavg2 = (maleprice.groupby(level=0).sum()).sum() / malenames

print(maleprice.groupby(level=0).sum())
    
    
#Females
isFemale = df['Gender']=='Female'
df_isFemale = df[isFemale]
femalesales = len(df_isFemale)
femaleavg = (df_isFemale['Price'].sum() / femalesales)
femalerevenue = df_isFemale['Price'].sum()

females1 = df_isFemale.sort_values(by=['SN'])
uniquefemales2 = pd.Series(df_isFemale['SN']).unique()   
uniquefemales2.sort()
femalenames = len(uniquefemales2)

females1 = females1.drop(columns=['Age', 'Gender', 'Item ID', 'Item Name'])
females1 = females1.reset_index(drop=True)
femaleprice = pd.Series(females1["Price"].values, index=females1['SN'])
femaleavg2 = (femaleprice.groupby(level=0).sum()).sum() / femalenames

print(femaleprice.groupby(level=0).sum())


#Other/Non-Disclosed
isOther = df['Gender']=='Other / Non-Disclosed'
df_isOther = df[isOther]
othersales = len(df_isOther)
otheravg = (df_isOther['Price'].sum() / othersales)
otherrevenue = df_isOther['Price'].sum()

other1 = df_isOther.sort_values(by=['SN'])
uniqueother2 = pd.Series(df_isOther['SN']).unique()   
uniqueother2.sort()
othernames = len(uniqueother2)

other1 = other1.drop(columns=['Age', 'Gender', 'Item ID', 'Item Name'])
other1 = other1.reset_index(drop=True)
otherprice = pd.Series(other1["Price"].values, index=other1['SN'])
otheravg2 = (otherprice.groupby(level=0).sum()).sum() / othernames

print((otherprice.groupby(level=0).sum()))



#Age Demographics
df3 = df3.drop(columns=['Item ID', 'Item Name'])
df3 = df3.sort_values(by=['Age'])
df3 = df3.reset_index(drop=True)
agebins = pd.cut(df3['Age'], 9)

ageseries = pd.Series(df3["Price"].values, index=agebins)
nameseries = pd.Series(df3['SN'].values, index=agebins)



      


agesum = ageseries.groupby(level=0).sum()
ageavg = ageseries.groupby(level=0).mean()
agesales = nameseries.groupby(level=0).count()
agesales = agesales.reset_index(drop=True)
agetotal = agesales.array


nameseries = nameseries.drop_duplicates()
agenum = nameseries.groupby(level=0).count()

ageavg2 = agesum / agenum

#Generate Text Report With Summary Statistics

f = open("Output/Summary.txt", "a")

f.write("\n" + "\n" + "Summary Stats \n")


f.write("Total Sales : " + str(totalsales) + "\n")
f.write("Total Players: " + str(totalplayers) + "\n")
f.write("Unique Items: " + str(uniqueitemsnum) + "\n")
f.write("Total Revenue: $" + str(totalrevenue) + "\n")
f.write("Average Price: $" + str(avgprice) + "\n")


f.close()



#Load Gender Data
genderdata = [(males, females, other), (malep, femalep, otherp), (malesales, femalesales, othersales), (malerevenue, femalerevenue, otherrevenue),
             (maleavg, femaleavg, otheravg), (maleavg2, femaleavg2, otheravg2)]


dfgender = pd.DataFrame(genderdata, columns = ['Male' , 'Female', 'Other'], index=['No. Players', '% Players', 'No. Sales',
                                                                                   '$ Total' , '$ Average Sale', '$ Average Per Person'])

#Load Age Data Into Nerw DataFrame


agedata = [agetotal, ageavg, agesum, ageavg2]

dfage = pd.DataFrame(agedata, columns = ['7-11', '12-15', '16-19', '20-23', '24-28', '29-32', '33-36', '37-40', '41-45' ], index=['No. Sales', '$ Avg Sale', 'Total $', '$ Avg Per Person'])


#Output
dfgender.to_csv(output_path)

dfage.to_csv(output_path2)



SN
Adairialis76    2.28
Aeda94          4.91
Aela59          4.32
Aelaria33       1.79
Aelastirin39    7.29
                ... 
Yasur85         1.10
Yathecal72      3.46
Yathedeu43      6.02
Zhisrisu83      7.89
Zontibe81       8.03
Length: 484, dtype: float64
SN
Adastirin33      4.48
Aerithllora36    8.64
Aethedru70       3.54
Aidain51         3.45
Aiduesu86        4.48
                 ... 
Wailin72         2.73
Yana46           3.55
Yarithrin84      4.14
Yathecal82       6.22
Yoishirrala98    4.58
Length: 81, dtype: float64
SN
Airi27          3.45
Chanosian48     7.52
Eodaisu60       3.94
Haerithp41      4.40
Idairin51       7.00
Jiskirran77     3.39
Lirtim36        1.33
Maluncil97      5.28
Rairith81       2.22
Siarithria38    6.91
Sundim98        4.75
dtype: float64
